In [ ]:
from transformers import AutoTokenizer, AutoModel

text = "What is Huggingface Transformers?"
# BERT
model_ckpt = "bert-base-uncased"
bert_model = AutoModel.from_pretrained(model_ckpt)
bert_tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
# PyTorch 이므로 return tensor를 pt로 
encoded_input = bert_tokenizer(text, return_tensors='pt')
bert_output = bert_model(**encoded_input)

c:\Users\user\anaconda3\envs\transformer_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## HuggingFace에서 RoBERTa

In [3]:
from transformers import AutoModel, AutoTokenizer

# 분류 헤드 렌덤으로 초기화된 모델 -> 파인튜닝 필수
model_ckpt = 'klue/roberta-base'
model = AutoModel.from_pretrained(model_ckpt) 

# 분류 헤드가 포함된 모델 
from transformers import AutoModelForSequenceClassification
model_id = 'SamLowe/roberta-base-go_emotions' # Down Stream(emotion)관련 분류기 
classification_model = AutoModelForSequenceClassification.from_pretrained(model_id)

c:\Users\user\anaconda3\envs\transformer_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--klue--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are ne

KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_ckpt = 'klue/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenized = tokenizer("토크나이저는 텍스트를 토큰 단위로 나눈다")
print(tokenized)
# input_ids는 각 토큰이 id로 변환된 값
# token_type_ids는 두개 이상의 문장이 입력으로 주어졌을 때 각 토큰이 어떤 문장에 속해있는지
# attention_mask는 토큰이 원본 문장에 존재하면 1, padding된 토큰들은 0

{'input_ids': [0, 9157, 7461, 2190, 2259, 8509, 2138, 1793, 2855, 5385, 2200, 20950, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
# 토크나이저는 input_ids를 토큰으로 재변환 가능 
print(tokenizer.convert_ids_to_tokens(tokenized['input_ids']))

['[CLS]', '토크', '##나이', '##저', '##는', '텍스트', '##를', '토', '##큰', '단위', '##로', '나눈다', '[SEP]']


In [10]:
# BERT기반 모델은 [CLS]토큰이 무조건 맨 앞에 위치한다. 
# 보퉁 A, B 두 개의 모델을 받아 NSP로 사전학습 되어있으므로, 문장을 분리할 [SEP] 토큰이 필요하다.
print(tokenizer.decode(tokenized['input_ids']))

[CLS] 토크나이저는 텍스트를 토큰 단위로 나눈다 [SEP]


In [11]:
print(tokenizer.decode(tokenized['input_ids'], skip_special_tokens=True))

토크나이저는 텍스트를 토큰 단위로 나눈다


## 토크나이저에 여러 문장 넣기

In [13]:
tokenizer(['첫 번째 문장', '두 번째 문장'])

{'input_ids': [[0, 1656, 1141, 3135, 6265, 2], [0, 864, 1141, 3135, 6265, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [14]:
first_tokenized_result = tokenizer(['첫 번째 문장','두 번째 문장'])['input_ids']
tokenizer.batch_decode(first_tokenized_result)

['[CLS] 첫 번째 문장 [SEP]', '[CLS] 두 번째 문장 [SEP]']